In [4]:
import os

base_path = "/home/wangsheng/Downloads/"
layer_path = os.path.join(base_path, "d838ff3af8f680812febc8a20a83f79e7c6126d8e57d3a9c26e6c31ad70f839a.pkl")
model_path = os.path.join(base_path, "glasses_efficientnet_b0.bin")

layer_size = os.path.getsize(layer_path)
model_size = os.path.getsize(model_path)
layer_sizes = layer_size*48

print(layer_size,layer_sizes, model_size)

result = layer_sizes/model_size
print(result)



385 18480 21456689
0.0008612698818536262


In [5]:
import os

base_path = "/home/wangsheng/Downloads/"
layer_path = os.path.join(base_path, "4815b6fdd87e7cd95f14168ed5602842b7f4c14b0f50d57c55a08d5d8e79481d.pkl")
model_path = os.path.join(base_path, "glasses_efficientnet_b2.bin")

layer_size = os.path.getsize(layer_path)
model_size = os.path.getsize(model_path)
layer_sizes = layer_size*68

print(layer_size,layer_sizes, model_size)

result = layer_sizes/model_size
print(result)

385 26180 36899721
0.0007094904592910066


In [6]:
import os

base_path = "/home/wangsheng/Downloads/"
layer_path = os.path.join(base_path, "6a50176d7f8e4b51d6363d0e48d2c26ccdc02900f60945afcb72dc20baadc184.pkl")
model_path = os.path.join(base_path, "glasses_efficientnet_b3.bin")

layer_size = os.path.getsize(layer_path)
model_size = os.path.getsize(model_path)
layer_sizes = layer_size*78

print(layer_size,layer_sizes, model_size)

result = layer_sizes/model_size
print(result)

385 30030 49495749
0.0006067187709392982


In [ ]:
import os

base_path = "/home/wangsheng/Downloads/"
layer_path1 = os.path.join(base_path, "57fffaac5f239cdc09c4e15dd08ff1eaae7aa0f0645c8837dcdc30c9b4da8d28.pkl")
layer_path2 = os.path.join(base_path, "4815b6fdd87e7cd95f14168ed5602842b7f4c14b0f50d57c55a08d5d8e79481d.pkl")
model_path = os.path.join(base_path, "glasses_efficientnet_b6.bin")

layer_size1 = os.path.getsize(layer_path1)

model_size = os.path.getsize(model_path)

layer_sizes = layer_size1*3+layer_size1*78

print(layer_size,layer_sizes, model_size)

result = layer_sizes/model_size
print(result)

385 30030 49495749
0.0006067187709392982


In [6]:
import subprocess
import hashlib

def get_image_layers(image, platform=None):
    command = ['docker', 'history', '--no-trunc', '--format', '{{.CreatedBy}}', image]
    result = subprocess.run(command, capture_output=True, text=True)
    return result.stdout.strip().split('\n')

def hash_layer(layer):
    return hashlib.sha256(layer.encode('utf-8')).hexdigest()

def calculate_similarity(layers1, layers2):
    hashes1 = set(hash_layer(layer) for layer in layers1)
    hashes2 = set(hash_layer(layer) for layer in layers2)
    common_hashes = hashes1.intersection(hashes2)
    total_layers = max(len(layers1), len(layers2))
    similarity = len(common_hashes) / total_layers
    return similarity

# 获取镜像层信息
amd64_layers = get_image_layers('nginx:latest')
arm64_layers = get_image_layers('192.168.0.9:5000/nginx:latest')

# 计算相似度
similarity = calculate_similarity(amd64_layers, arm64_layers)

print(f"Similarity between nginx:amd64 and nginx:arm64: {similarity:.2%}")


Similarity between nginx:amd64 and nginx:arm64: 94.12%


In [1]:
import subprocess
import hashlib

def get_image_layers(image, platform=None):
    command = ['docker', 'history', '--no-trunc', '--format', '{{.CreatedBy}}', image]
    if platform:
        command.extend(['--platform', platform])
    result = subprocess.run(command, capture_output=True, text=True)
    return result.stdout.strip().split('\n')

def hash_layer(layer):
    return hashlib.sha256(layer.encode('utf-8')).hexdigest()

def calculate_similarity(layers1, layers2):
    hashes1 = {hash_layer(layer): layer for layer in layers1}
    hashes2 = {hash_layer(layer): layer for layer in layers2}
    
    common_hashes = set(hashes1.keys()).intersection(hashes2.keys())
    total_layers = max(len(layers1), len(layers2))
    similarity = len(common_hashes) / total_layers
    
    different_layers1 = [hashes1[h] for h in hashes1 if h not in common_hashes]
    different_layers2 = [hashes2[h] for h in hashes2 if h not in common_hashes]
    
    return similarity, different_layers1, different_layers2

# 获取镜像层信息
amd64_layers = get_image_layers('nginx:amd64')
arm64_layers = get_image_layers('nginx:arm64')

# 计算相似度和不同的层信息
similarity, different_layers1, different_layers2 = calculate_similarity(amd64_layers, arm64_layers)

print(f"Similarity between nginx:amd64 and nginx:arm64: {similarity:.2%}")
print("\nLayers in amd64 not in arm64:")
for layer in different_layers1:
    print(layer)

print("\nLayers in arm64 not in amd64:")
for layer in different_layers2:
    print(layer)


Similarity between nginx:amd64 and nginx:arm64: 95.00%

Layers in amd64 not in arm64:
/bin/sh -c #(nop) ADD file:37a76ec18f9887751cd8473744917d08b7431fc4085097bb6a09d81b41775473 in /

Layers in arm64 not in amd64:
/bin/sh -c #(nop) ADD file:d0764a717d1e9d0aff3fa84779b11bfa0afe4430dcb6b46d965b209167639ba0 in /


In [16]:
def read_file(file_path):
    """读取文件内容并返回每行作为列表项"""
    with open(file_path, 'r') as file:
        return file.readlines()

# 读取两个文本文件的内容
file1_lines = read_file('arm64_output.txt')
file2_lines = read_file('amd64_output.txt')

# 从每行中提取with hash后的部分和文件大小
def extract_hash_and_size(line):
    parts = line.split('with hash')[1].strip().split(', size:')
    hash_value = parts[0].strip()
    size = int(parts[1].split()[0])  # 提取文件大小并转换为整数
    return hash_value, size

# 从文件内容中提取哈希值和文件大小的字典
file1_hashes_and_sizes = {extract_hash_and_size(line)[0]: extract_hash_and_size(line)[1] for line in file1_lines}
file2_hashes_and_sizes = {extract_hash_and_size(line)[0]: extract_hash_and_size(line)[1] for line in file2_lines}

# 找出两个文件共同的哈希值
common_hashes = set(file1_hashes_and_sizes.keys()).intersection(file2_hashes_and_sizes.keys())

# 计算共同层的大小总和
total_size_file1 = 0
total_size_file2 = 0

for hash_value in common_hashes:
    total_size_file1 += file1_hashes_and_sizes[hash_value]
    total_size_file2 += file2_hashes_and_sizes[hash_value]

# 输出共同的哈希值对应的行及其大小
print("共同的项目:")
for hash_value in common_hashes:
    for line in file1_lines:
        if hash_value in line:
            print(line.strip())
            break
    for line in file2_lines:
        if hash_value in line:
            print(line.strip())
            break

print("\n共同项目的大小总和:")
print("arm64文件大小总和:", total_size_file1, "bytes")
print("amd64文件大小总和:", total_size_file2, "bytes")
print("arm64冗余:", total_size_file1 / 62390207)
print("amd64冗余:", total_size_file2 / 60817408)

共同的项目:
Processed file: /var/lib/docker/overlay2/a7abc557d37e2261bd5bc4e19a45bfae3b08a3561242aa42811f0bd00c53b957/diff/usr/share/zoneinfo/Asia/Qostanay with hash 1c88e59fc4083cd454e8c916cf23e2eaf0eb8e59956785f31717141ed5cc8bbb, size: 1025 bytes
Processed file: /var/lib/docker/overlay2/5897a9738141c5b93fe128d512e624bc37923b16d15f91d209628fc5102cdf49/diff/usr/share/zoneinfo/Asia/Qostanay with hash 1c88e59fc4083cd454e8c916cf23e2eaf0eb8e59956785f31717141ed5cc8bbb, size: 1025 bytes
Processed file: /var/lib/docker/overlay2/a4e58ce122964a02ec8af760effdc0f8cfa57215f10d1e98d822eac33da083cd/diff/etc/terminfo/s/sun with hash 02e392161cb23f49a8fb1ba2f1a6583e013c0c26672f58c5eaca828db3b19914, size: 1004 bytes
Processed file: /var/lib/docker/overlay2/3c595fbec79b4128c3efbb2c2dc1988615d36eca857b69c81a0d6f56b93ab057/diff/etc/terminfo/s/sun with hash 02e392161cb23f49a8fb1ba2f1a6583e013c0c26672f58c5eaca828db3b19914, size: 1004 bytes
Processed file: /var/lib/docker/overlay2/a4e58ce122964a02ec8af760effdc0f8

In [17]:
def read_file(file_path):
    """读取文件内容并返回每行作为列表项"""
    with open(file_path, 'r') as file:
        return file.readlines()

# 读取两个文本文件的内容
file1_lines = read_file('arm64_output.txt')
file2_lines = read_file('amd64_output.txt')

# 从每行中提取with hash后的部分和文件大小
def extract_hash_and_size(line):
    parts = line.split('with hash')[1].strip().split(', size:')
    hash_value = parts[0].strip()
    size = int(parts[1].split()[0])  # 提取文件大小并转换为整数
    return hash_value, size

# 从文件内容中提取哈希值和文件大小的字典
file1_hashes_and_sizes = {extract_hash_and_size(line)[0]: extract_hash_and_size(line)[1] for line in file1_lines}
file2_hashes_and_sizes = {extract_hash_and_size(line)[0]: extract_hash_and_size(line)[1] for line in file2_lines}

# 找出每个文件的所有哈希值
all_hashes_file1 = set(file1_hashes_and_sizes.keys())
all_hashes_file2 = set(file2_hashes_and_sizes.keys())

# 找出共同的哈希值
common_hashes = all_hashes_file1.intersection(all_hashes_file2)

# 计算共同哈希值的大小总和
total_size_file1 = sum(file1_hashes_and_sizes.get(hash_value, 0) for hash_value in common_hashes)
total_size_file2 = sum(file2_hashes_and_sizes.get(hash_value, 0) for hash_value in common_hashes)

# 输出共同的哈希值对应的行及其大小
print("共同的项目:")
for hash_value in common_hashes:
    found_in_file1 = False
    found_in_file2 = False
    for line in file1_lines:
        if hash_value in line:
            print(line.strip())
            found_in_file1 = True
            break
    for line in file2_lines:
        if hash_value in line:
            print(line.strip())
            found_in_file2 = True
            break
    # 如果在其中一个文件中找不到，则打印空行
    if not found_in_file1:
        print()
    if not found_in_file2:
        print()

print("\n共同项目的大小总和:")
print("arm64文件大小总和:", total_size_file1, "bytes")
print("amd64文件大小总和:", total_size_file2, "bytes")
print("arm64冗余:", total_size_file1 / 62390207)
print("amd64冗余:", total_size_file2 / 60817408)


共同的项目:
Processed file: /var/lib/docker/overlay2/a7abc557d37e2261bd5bc4e19a45bfae3b08a3561242aa42811f0bd00c53b957/diff/usr/share/zoneinfo/Asia/Qostanay with hash 1c88e59fc4083cd454e8c916cf23e2eaf0eb8e59956785f31717141ed5cc8bbb, size: 1025 bytes
Processed file: /var/lib/docker/overlay2/5897a9738141c5b93fe128d512e624bc37923b16d15f91d209628fc5102cdf49/diff/usr/share/zoneinfo/Asia/Qostanay with hash 1c88e59fc4083cd454e8c916cf23e2eaf0eb8e59956785f31717141ed5cc8bbb, size: 1025 bytes
Processed file: /var/lib/docker/overlay2/a4e58ce122964a02ec8af760effdc0f8cfa57215f10d1e98d822eac33da083cd/diff/etc/terminfo/s/sun with hash 02e392161cb23f49a8fb1ba2f1a6583e013c0c26672f58c5eaca828db3b19914, size: 1004 bytes
Processed file: /var/lib/docker/overlay2/3c595fbec79b4128c3efbb2c2dc1988615d36eca857b69c81a0d6f56b93ab057/diff/etc/terminfo/s/sun with hash 02e392161cb23f49a8fb1ba2f1a6583e013c0c26672f58c5eaca828db3b19914, size: 1004 bytes
Processed file: /var/lib/docker/overlay2/a7abc557d37e2261bd5bc4e19a45bfae

In [19]:
def read_file(file_path):
    """读取文件内容并返回每行作为列表项"""
    with open(file_path, 'r') as file:
        return file.readlines()

# 读取两个文本文件的内容
file1_lines = read_file('arm64_output.txt')
file2_lines = read_file('amd64_output.txt')

# 从每行中提取with hash后的部分和文件大小
def extract_hash_and_size(line):
    parts = line.split('with hash')[1].strip().split(', size:')
    hash_value = parts[0].strip()
    size = int(parts[1].split()[0])  # 提取文件大小并转换为整数
    return hash_value, size

# 从文件内容中提取哈希值和文件大小的字典
file1_hashes_and_sizes = {extract_hash_and_size(line)[0]: (line.strip(), extract_hash_and_size(line)[1]) for line in file1_lines}
file2_hashes_and_sizes = {extract_hash_and_size(line)[0]: (line.strip(), extract_hash_and_size(line)[1]) for line in file2_lines}

# 找出每个文件的所有哈希值
all_hashes_file1 = set(file1_hashes_and_sizes.keys())
all_hashes_file2 = set(file2_hashes_and_sizes.keys())

# 找出共同的哈希值
common_hashes = all_hashes_file1.intersection(all_hashes_file2)

# 计算共同哈希值的大小总和
total_size_file1 = sum(file1_hashes_and_sizes[hash_value][1] for hash_value in common_hashes)
total_size_file2 = sum(file2_hashes_and_sizes[hash_value][1] for hash_value in common_hashes)

# 输出共同的哈希值对应的行及其大小
print("共同的项目:")
for hash_value in common_hashes:
    file1_line, size1 = file1_hashes_and_sizes.get(hash_value, ("", 0))
    file2_line, size2 = file2_hashes_and_sizes.get(hash_value, ("", 0))
    if file1_line:
        print(file1_line)
    if file2_line:
        print(file2_line)

print("\n共同项目的大小总和:")
print("arm64文件大小总和:", total_size_file1, "bytes")
print("amd64文件大小总和:", total_size_file2, "bytes")
print("arm64冗余:", total_size_file1 / 62390207)
print("amd64冗余:", total_size_file2 / 60817408)


共同的项目:
Processed file: /var/lib/docker/overlay2/a7abc557d37e2261bd5bc4e19a45bfae3b08a3561242aa42811f0bd00c53b957/diff/usr/share/zoneinfo/Asia/Qostanay with hash 1c88e59fc4083cd454e8c916cf23e2eaf0eb8e59956785f31717141ed5cc8bbb, size: 1025 bytes
Processed file: /var/lib/docker/overlay2/5897a9738141c5b93fe128d512e624bc37923b16d15f91d209628fc5102cdf49/diff/usr/share/zoneinfo/Asia/Qostanay with hash 1c88e59fc4083cd454e8c916cf23e2eaf0eb8e59956785f31717141ed5cc8bbb, size: 1025 bytes
Processed file: /var/lib/docker/overlay2/a4e58ce122964a02ec8af760effdc0f8cfa57215f10d1e98d822eac33da083cd/diff/etc/terminfo/s/sun with hash 02e392161cb23f49a8fb1ba2f1a6583e013c0c26672f58c5eaca828db3b19914, size: 1004 bytes
Processed file: /var/lib/docker/overlay2/3c595fbec79b4128c3efbb2c2dc1988615d36eca857b69c81a0d6f56b93ab057/diff/etc/terminfo/s/sun with hash 02e392161cb23f49a8fb1ba2f1a6583e013c0c26672f58c5eaca828db3b19914, size: 1004 bytes
Processed file: /var/lib/docker/overlay2/a7abc557d37e2261bd5bc4e19a45bfae

In [21]:
def read_file(file_path):
    """读取文件内容并返回每行作为列表项"""
    with open(file_path, 'r') as file:
        return file.readlines()

# 读取两个文本文件的内容
file1_lines = read_file('zookeeper_output.txt')
file2_lines = read_file('ubuntu_zookeeper_output.txt')

# 从每行中提取with hash后的部分和文件大小
def extract_hash_and_size(line):
    parts = line.split('with hash')[1].strip().split(', size:')
    hash_value = parts[0].strip()
    size = int(parts[1].split()[0])  # 提取文件大小并转换为整数
    return hash_value, size

# 从文件内容中提取哈希值和文件大小的字典
file1_hashes_and_sizes = {extract_hash_and_size(line)[0]: (line.strip(), extract_hash_and_size(line)[1]) for line in file1_lines}
file2_hashes_and_sizes = {extract_hash_and_size(line)[0]: (line.strip(), extract_hash_and_size(line)[1]) for line in file2_lines}

# 找出每个文件的所有哈希值
all_hashes_file1 = set(file1_hashes_and_sizes.keys())
all_hashes_file2 = set(file2_hashes_and_sizes.keys())

# 找出共同的哈希值
common_hashes = all_hashes_file1.intersection(all_hashes_file2)

# 计算共同哈希值的大小总和
total_size_file1 = sum(file1_hashes_and_sizes[hash_value][1] for hash_value in common_hashes)
total_size_file2 = sum(file2_hashes_and_sizes[hash_value][1] for hash_value in common_hashes)

# 输出共同的哈希值对应的行及其大小
print("共同的项目:")
for hash_value in common_hashes:
    file1_line, size1 = file1_hashes_and_sizes.get(hash_value, ("", 0))
    file2_line, size2 = file2_hashes_and_sizes.get(hash_value, ("", 0))
    if file1_line:
        print(file1_line)
    if file2_line:
        print(file2_line)

print("\n共同项目的大小总和:")
print("zookeeper_output文件大小总和:", total_size_file1, "bytes")
print("ubuntu_zookeeper_output文件大小总和:", total_size_file2, "bytes")
print("zookeeper_output冗余:", total_size_file1 / 328448288)
print("ubuntu_zookeeper_output冗余:", total_size_file2 / 401920768)


共同的项目:
Processed file: /var/lib/docker/overlay2/9d0ab7e950491e9680e02d9c28e0a25e5237c84a024e9e0dfeb402667f9c425a/diff/usr/share/sensible-utils/bin/gettext with hash f653ffb26dc99dc1a0a931e08176195528aa9415d5085967a9d8169821a82875, size: 63 bytes
Processed file: /var/lib/docker/overlay2/5c1a62f47360dd02a0cf9451b0cc184639aed203e520fb24800f6f81010661ca/diff/usr/share/sensible-utils/bin/gettext with hash f653ffb26dc99dc1a0a931e08176195528aa9415d5085967a9d8169821a82875, size: 63 bytes
Processed file: /var/lib/docker/overlay2/9d0ab7e950491e9680e02d9c28e0a25e5237c84a024e9e0dfeb402667f9c425a/diff/usr/lib/x86_64-linux-gnu/perl-base/unicore/lib/Bc/B.pl with hash 0fc5b158814d575a5ed69928258dbe0ef31d24df6da2887b721fbad6332d156a, size: 527 bytes
Processed file: /var/lib/docker/overlay2/5c1a62f47360dd02a0cf9451b0cc184639aed203e520fb24800f6f81010661ca/diff/usr/lib/x86_64-linux-gnu/perl-base/unicore/lib/Bc/B.pl with hash 0fc5b158814d575a5ed69928258dbe0ef31d24df6da2887b721fbad6332d156a, size: 527 bytes